In [ ]:
from neo4j import GraphDatabase
import pandas as pd
import random

# Funzione per inizializzare la connessione al database Neo4j
def create_connection(uri, user, password):
    return GraphDatabase.driver(uri, auth=(user, password))

# Funzione per chiudere la connessione al database Neo4j
def close_connection(driver):
    driver.close()

# Funzione per creare il grafo nel database Neo4j
def create_graph(tx, utenti_data, posts_data, ingredienti_data):
    # Creazione dei nodi Utente
    for index, utente in utenti_data.iterrows():
        tx.run("CREATE (:User {_id: $id, username: $username})", id=utente['_id']['$oid'], username=utente['username'])

    # Creazione dei nodi Ricetta e collegamenti con Utente e Ingrediente
    for index, post in posts_data.iterrows():
        tx.run("CREATE (:Recipe {_id: $id, name: $name})", id=post['_id']['$oid'], name=post['recipe']['name'])

#        # Collegamento tra Ricetta e Utente
#        tx.run("MATCH (r:Ricetta {id: $ricetta_id}), (u:Utente {id: $utente_id}) CREATE (r)-[:HA_CREATO]->(u)", ricetta_id=ricetta['id'], utente_id=ricetta['utente'])

        # Collegamento tra Ricetta e Ingrediente e attributo conteggio
        for ingrediente in post['recipe']['ingredients']:
            tx.run("MATCH (r:Recipe {_id: $ricetta_id}), (i:Ingrediente {_id: $ingrediente_id}) CREATE (r)-[:CONTAINS]->(i)", ricetta_id=ricetta['id'], ingrediente_id=ingrediente['id'])

    # Collegamento tra Utenti (seguaci) in modo casuale
#    for utente in utenti_data:
#        utente_id2 = random.choice(utenti_data)['_id']['$oid']
#        bidirezionale = random.choice([True, False])
#
#        if bidirezionale:
#            tx.run("MATCH (u1:Utente {id: $utente_id1}), (u2:Utente {id: $utente_id2}) CREATE (u1)-[:SEGUE]->(u2), (u2)-[:È_SEGUITO_DA]->(u1)", utente_id1=utente['id'], utente_id2=utente_id2)
#        else:
#            tx.run("MATCH (u1:Utente {id: $utente_id1}), (u2:Utente {id: $utente_id2}) CREATE (u1)-[:SEGUE]->(u2)", utente_id1=utente['id'], utente_id2=utente_id2)

#   BISOGNA AGGIUNGERE COLLEGAMENTI TRA UTENTI CHE SI SEGUONO A VICENDA E UTENTI CON INGREDIENTI

    # Collegamento tra Ingredienti che compaiono insieme nelle ricette
    for post in posts_data:
        ricetta = post['recipe']
        for i, ingrediente1 in enumerate(ricetta['ingredients']):
            for j, ingrediente2 in enumerate(ricetta['ingredients']):
                if i != j:
                    tx.run("MATCH (i1:Ingredient {_id: $ingrediente_id1}), (i2:Ingredient {_id: $ingrediente_id2}) MERGE (i1)-[c:USED_WITH]->(i2) ON CREATE SET c.times = 1 ON MATCH SET c.times = c.times + 1", ingrediente_id1=ingrediente1['_id']['$oid'], ingrediente_id2=ingrediente2['_id']['$oid'])

# Configurazione della connessione al database Neo4j
uri = "bolt://localhost:7687"  # Cambia con l'URL del tuo database
user = "neo4j"
password = "password"  # Cambia con la password del tuo database

# Inizializzazione della connessione
driver = create_connection(uri, user, password)

utenti_data = pd.read_json('../users_database.json')
posts_data = pd.read_json('../posts_database.json')
ingredienti_data = pd.read_json('../ingredients.json', lines=True)

# Esecuzione del codice nel contesto di una transazione Neo4j
with driver.session() as session:
    session.write_transaction(create_graph, utenti_data, posts_data, ingredienti_data)

# Chiusura della connessione
close_connection(driver)

In [ ]:
from neo4j import GraphDatabase
import pandas as pd

# Connessione al Database Neo4j
uri = "bolt://10.1.1.24:7687"
user = "neo4j"
password = "password"

class Neo4jConnection:
    def __init__(self, uri, user, password):
        self.__uri = uri
        self.__user = user
        self.__password = password
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__password))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [ ]:
conn = Neo4jConnection(uri, user, password)

# Caricamento dei dati
utenti_data = pd.read_json('../users_database.json')
posts_data = pd.read_json('../posts_database.json')
ingredienti_data = pd.read_json('../ingredients.json', lines=True)

# Creazione dei nodi Utente
for index, row in utenti_data.iterrows():
    conn.query("CREATE (u:User {_id: $id, username: $username})", {"id": row["_id"]["$oid"], "username": row["username"]})
    
print("Nodi Utente creati")

# Creazione dei nodi Ricetta
for index, row in posts_data.iterrows():
    # Assumendo che ogni post abbia una ricetta innestata
    recipe = row["recipe"]  # o il campo appropriato che contiene la recipe
    recipe_id = row["_id"]["$oid"]
    recipe_name = recipe["name"]
    # Aggiungi altri campi della ricetta se necessario

    # Creazione del nodo Ricetta
    conn.query("CREATE (r:Recipe {_id: $recipe_id, name: $recipe_name})", {"recipe_id": recipe_id, "recipe_name": recipe_name})

print("Nodi Ricetta creati")

# Creazione dei nodi Ingrediente
for index, row in ingredienti_data.iterrows():
    conn.query("CREATE (i:Ingredient {_id: $id, name: $name})", {"id": row["_id"]["$oid"], "name": row["name"]})
    
print("Nodi Ingrediente creati")

# Chiusura della connessione
conn.close()


In [3]:
conn = Neo4jConnection(uri, user, password)

# Creazione dei nodi Ingrediente
for index, row in ingredienti_data.iterrows():
    conn.query("CREATE (i:Ingredient {_id: $id, name: $name})", {"id": row["_id"]["$oid"], "name": row["name"]})
    
print("Nodi Ingrediente creati")

# Chiusura della connessione
conn.close()


Nodi Ingrediente creati


Recipe -> :CONTAINS -> Ingredient

In [ ]:
conn = Neo4jConnection(uri, user, password)

utenti_data = pd.read_json('../users_database.json')
posts_data = pd.read_json('../posts_database.json')
ingredienti_data = pd.read_json('../ingredients.json', lines=True)

for index, row in posts_data.iterrows():
    recipe = row["recipe"]
    recipe_id = row["_id"]["$oid"]
    
    # Itera attraverso gli ingredienti nella ricetta
    for ingredient in recipe["ingredients"]:
        ingredient_name = ingredient["name"]
        
        # Crea una relazione tra Ricetta e Ingrediente
        conn.query("""
            MATCH (r:Recipe {_id: $recipe_id}), (i:Ingredient {name: $ingredient_name})
            MERGE (r)-[:CONTAINS]->(i)
            """, {"recipe_id": recipe_id, "ingredient_name": ingredient_name})

conn.close()

Ingredient -> :USED_WITH -> Ingredient

In [ ]:
conn = Neo4jConnection(uri, user, password)

posts_data = pd.read_json('../posts_database.json')
ingredienti_data = pd.read_json('../ingredients.json', lines=True)

for index, row in posts_data.iterrows():
    recipe = row["recipe"]
    
    for i, ingrediente1 in enumerate(recipe['ingredients']):
        for j, ingrediente2 in enumerate(recipe['ingredients']):
            if ingrediente1['name'] != ingrediente2['name']:
                print(ingrediente1['name'], ingrediente2['name'])
                conn.query("""
                        MATCH (i1:Ingredient {name: $ingredient1_name}), (i2:Ingredient {name: $ingredient2_name}) 
                        MERGE (i1)-[c:USED_WITH]->(i2) ON CREATE SET c.times = 1 ON MATCH SET c.times = c.times + 1
                        """, {"ingredient1_name": ingrediente1['name'], "ingredient2_name": ingrediente2['name']})
    
    # Itera attraverso gli ingredienti nella ricetta

conn.close()


User -> :USED -> Ingredient

In [ ]:
conn = Neo4jConnection(uri, user, password)

posts_data = pd.read_json('../posts_database.json')
ingredienti_data = pd.read_json('../ingredients.json', lines=True)

for index, row in posts_data.iterrows():
    recipe = row["recipe"]
    user_id = row["idUser"]["$oid"]
    
    for ingredient in recipe["ingredients"]:
        ingredient_name = ingredient["name"]
        
        # Crea una relazione tra Ricetta e Ingrediente
        conn.query("""
            MATCH (u:User {_id: $user_id}), (i:Ingredient {name: $ingredient_name})
            MERGE (u)-[r:USED]->(i) ON CREATE SET r.times = 1 ON MATCH SET r.times = r.times + 1
            """, {"user_id": user_id, "ingredient_name": ingredient_name})

    # Itera attraverso gli ingredienti nella ricetta

conn.close()


USER -> :FOLLOWS -> USER
If a user comment a post, we assume that he/she follows the author of the post

In [ ]:
conn = Neo4jConnection(uri, user, password)

posts_data = pd.read_json('../posts_database.json')

for index, row in posts_data.iterrows():
    if str(row["comments"]) == "nan":
        continue

    comments = row["comments"]
    author = row["idUser"]["$oid"]
    
    for comment in comments:
        user = comment["idUser"]["$oid"]
        
        if user == author:
            continue
        
        # Crea una relazione tra Ricetta e Ingrediente
        conn.query("""
            MATCH (u1:User {_id: $user}), (u2:User {_id: $author})
            MERGE (u1)-[:FOLLOWS]->(u2)
            """, {"user": user, "author": author})

conn.close()